In [2]:
using Revise
using Turing
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [3]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [4]:
includet("utils.jl")
includet("VAR_OLS.jl")

In [5]:
df = DataFrame(XLSX.readtable("../../data/usa_world_data.xlsx", "Arkusz1", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

146×15 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-03-01 to 2024-04-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1988-03-01 │   0.0857 │   677.401 │       262.6 │  9.04333 │  18.59 │ 14.433 ⋯
│ 1988-06-01 │   0.0882 │   681.206 │       269.1 │  9.30667 │  21.67 │ 14.608 ⋯
│ 1988-09-01 │   0.0887 │   694.387 │       277.4 │     9.55 │  22.73 │ 14.811 ⋯
│ 1988-12-01 │   0.0914 │   697.568 │       285.4 │  9.81333 │  23.75 │ 15.088 ⋯
│ 1989-03-01 │    0.093 │   706.377 │       302.3 │  10.0867 │  24.96 │ 15.686 ⋯
│ 1989-06-01 │    0.081 │    780.73 │       331.9 │  10.4233 │  25.22 │ 17.013 ⋯
│ 1989-09-01 │   0.0831 │   785.343 │       347.4 │  10.7967 │  23.69 │ 17.640 ⋯
│ 1989-12-01 │   0.0793 │   821.683 │      339.97 │    11.14 │  22.87 │ 17.048 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [6]:
freq = 4
returns = percentchange(data_source[:ACWI_Net, :USA_Net, :Price_10Y, :World_ex_USA, :EM_Net], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
real_10Y = log1p.(data_source[:Yeld_10Y])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
dy = log.(data_source[:Dividend_Yeld])
term = log1p.(data_source[:Yeld_10Y] ./ freq)  .- log1p.(data_source[:T_Bill] ./ freq)
term = data_source[:Yeld_10Y]   .- log1p.(data_source[:T_Bill] 
#data = merge(real_returns, real_t_bill, term, cpi, dy)
data = real_returns

start = Date(1989, 01,01)
end_d = Date(2019, 12, 31)
s_data = to(from(data, start),end_d)

display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))
display("Excess_returns")
display(returns_summarystats(to(from(excess_returns, start),end_d), freq))

display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

Base.Meta.ParseError: ParseError:
# Error @ c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner\_wip\VARs\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W4sZmlsZQ==.jl:12:1
#data = merge(real_returns, real_t_bill, term, cpi, dy)
┌──────────────────
data = real_returns

start = Date(1989, 01,01)
#──────────────────────┘ ── Expected `)`

In [7]:
display(returns_summarystats(to(from(merge(real_t_bill, real_10Y, dy, term), start),end_d), freq))

UndefVarError: UndefVarError: `start` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [8]:
start = Date(1989, 01, 01)
end_d = Date(2024, 01, 31)
assets_names = [:USA_Net_T_Bill, :World_ex_USA_T_Bill, :EM_Net_T_Bill]
data = to(from(merge(excess_returns[assets_names], real_t_bill, dy, term), start),end_d)
assets_names = colnames(data)
T, n_assets = size(data)
model = VARModel(data)
model_summary(model)

,const,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill,std
USA_Net_T_Bill,0.1697,0.1248,0.0204,-0.0203,-1.6295,0.0354,-3.9738,0.0724
World_ex_USA_T_Bill,-0.0233,0.0263,0.018,0.0578,-0.9841,-0.0081,-1.233,0.0845
EM_Net_T_Bill,0.2301,-0.2145,0.2693,0.0243,-0.2681,0.0531,-1.6701,0.1275
T_Bill_CPI,0.0274,-0.0004,0.0095,-0.0027,0.2479,0.0063,-0.6251,0.0081
Dividend_Yeld,-0.2257,0.0248,-0.1052,0.0341,-0.9466,0.947,2.7692,0.0736
Yeld_10Y_T_Bill,0.0022,-0.0017,-0.0011,0.0014,-0.0069,0.0005,0.9021,0.0012


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill
USA_Net_T_Bill,1.0,0.8,0.65,-0.13,-0.59,0.21
World_ex_USA_T_Bill,0.8,1.0,0.74,-0.26,-0.48,0.18
EM_Net_T_Bill,0.65,0.74,1.0,-0.26,-0.32,0.31
T_Bill_CPI,-0.13,-0.26,-0.26,1.0,0.08,-0.2
Dividend_Yeld,-0.59,-0.48,-0.32,0.08,1.0,-0.08
Yeld_10Y_T_Bill,0.21,0.18,0.31,-0.2,-0.08,1.0


In [9]:
scenarios = simulate(model, 100, 10_000)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.03, .25, .5, .75, .97], string.(periods), string.(assets_names[a]))
end  

,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill
1,0.0722,0.0229,0.012,0.0221,-16.9944,-0.0107
5,0.0804,0.0399,0.0045,0.0069,-17.1185,-0.0027
15,0.071,0.0393,0.0084,0.001,-16.946,0.0039
25,0.0686,0.0369,0.0129,-0.0001,-16.7963,0.0063


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill
1,0.1373,0.1606,0.2321,0.017,0.2683,0.0043
5,0.0704,0.0867,0.1175,0.0115,0.5668,0.0078
15,0.0402,0.0536,0.0709,0.0076,0.6932,0.0078
25,0.031,0.0424,0.0564,0.006,0.6684,0.0071


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill
1,-0.0115,0.024,0.0026,0.0238,0.0412,0.004
5,0.0156,-0.0246,0.0043,0.0327,-0.0016,-0.0005
15,0.0045,0.0043,-0.0132,0.0469,0.0082,0.0262
25,-0.0065,-0.0057,-0.0028,0.0676,0.0259,0.0272


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill
1,-0.0268,0.0171,0.0,0.0547,-0.019,-0.077
5,-0.0421,-0.0383,-0.0469,0.0169,0.1231,-0.0284
15,-0.0092,-0.0277,-0.0354,-0.0359,0.0479,0.0372
25,-0.0276,-0.0075,-0.0724,0.0538,0.0568,0.0186


,0.03,0.25,0.5,0.75,0.97
1,-0.1827,-0.0218,0.0709,0.1656,0.3284
5,-0.052,0.0322,0.0805,0.1286,0.2133
15,-0.0044,0.0438,0.0711,0.0982,0.147
25,0.0107,0.0478,0.0686,0.0893,0.1273


,0.03,0.25,0.5,0.75,0.97
1,-0.2747,-0.0849,0.0238,0.1287,0.327
5,-0.1228,-0.0179,0.0402,0.0984,0.2013
15,-0.0622,0.0037,0.0388,0.0758,0.1404
25,-0.0438,0.0089,0.0371,0.0652,0.1165


,0.03,0.25,0.5,0.75,0.97
1,-0.424,-0.1416,0.0124,0.1655,0.4523
5,-0.2182,-0.0753,0.0037,0.0841,0.2256
15,-0.1253,-0.0402,0.0079,0.057,0.1397
25,-0.0927,-0.025,0.0132,0.0512,0.1177


,0.03,0.25,0.5,0.75,0.97
1,-0.0099,0.0109,0.022,0.0335,0.0536
5,-0.0146,-0.0008,0.007,0.0145,0.0286
15,-0.0132,-0.0042,0.001,0.0062,0.0155
25,-0.0114,-0.0042,-0.0002,0.0038,0.0114


,0.03,0.25,0.5,0.75,0.97
1,-17.4923,-17.1769,-16.9944,-16.8156,-16.4888
5,-18.1922,-17.4906,-17.1212,-16.7393,-16.0588
15,-18.2401,-17.4186,-16.9422,-16.4735,-15.6538
25,-18.0415,-17.246,-16.8012,-16.3507,-15.5238


,0.03,0.25,0.5,0.75,0.97
1,-0.0188,-0.0137,-0.0107,-0.0078,-0.0025
5,-0.0172,-0.0079,-0.0027,0.0027,0.0122
15,-0.0105,-0.0015,0.0039,0.0091,0.0185
25,-0.0072,0.0015,0.0062,0.011,0.0197


In [10]:
start = Date(1989, 01, 01)
end_d = Date(2024, 01, 31)
assets_names = [:USA_Net_T_Bill, :Price_10Y_T_Bill, :World_ex_USA_T_Bill, :EM_Net_T_Bill]
data = to(from(merge(excess_returns[assets_names], real_t_bill, dy), start),end_d)
assets_names = colnames(data)
T, n_assets = size(data)
model = VARModel(data)
model_summary(model)

,const,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,std
USA_Net_T_Bill,0.0895,0.1488,0.1377,0.0134,-0.0154,-1.3644,0.0193,0.073
Price_10Y_T_Bill,0.0639,0.077,-0.0663,-0.0155,-0.0312,0.6806,0.0147,0.0413
World_ex_USA_T_Bill,-0.0463,0.0307,0.0167,0.0169,0.0567,-0.8777,-0.0126,0.0846
EM_Net_T_Bill,0.1904,-0.1945,0.1415,0.2628,0.0347,-0.2343,0.045,0.1275
T_Bill_CPI,0.0184,-0.0026,-0.0287,0.0105,-0.007,0.3363,0.0046,0.0082
Dividend_Yeld,-0.1668,0.0032,-0.1374,-0.0985,0.0265,-1.0929,0.9588,0.0738


,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld
USA_Net_T_Bill,1.0,-0.38,0.8,0.65,-0.09,-0.59
Price_10Y_T_Bill,-0.38,1.0,-0.36,-0.43,0.25,0.2
World_ex_USA_T_Bill,0.8,-0.36,1.0,0.74,-0.25,-0.48
EM_Net_T_Bill,0.65,-0.43,0.74,1.0,-0.25,-0.32
T_Bill_CPI,-0.09,0.25,-0.25,-0.25,1.0,0.05
Dividend_Yeld,-0.59,0.2,-0.48,-0.32,0.05,1.0


In [11]:
scenarios = simulate(model, 100, 10_000)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.03, .25, .5, .75, .97], string.(periods), string.(assets_names[a]))
end  

,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld
1,0.0251,0.0638,0.0124,0.0041,0.0085,-16.912
5,0.0462,0.02,0.0276,0.0086,-0.0026,-16.7139
15,0.0539,0.0165,0.0281,0.0196,-0.0032,-16.4748
25,0.0561,0.0166,0.0276,0.0234,-0.0031,-16.3841


,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld
1,0.1373,0.0716,0.1603,0.2328,0.0179,0.2729
5,0.0698,0.0377,0.0867,0.1179,0.0101,0.5724
15,0.04,0.0237,0.0543,0.0705,0.0064,0.6062
25,0.0303,0.019,0.0426,0.0546,0.0051,0.5371


,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld
1,-0.023,0.0015,-0.0137,0.0122,-0.0278,0.0092
5,0.0006,-0.0295,0.014,-0.0044,-0.0371,-0.0408
15,-0.0425,-0.0007,-0.0289,0.0117,0.0242,0.0091
25,-0.0131,-0.023,-0.0112,0.0132,0.0305,0.002


,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld
1,0.0501,0.0363,0.0336,0.0149,0.004,-0.0396
5,-0.0026,0.0336,0.0156,0.0225,0.0395,-0.0018
15,0.0048,0.0671,-0.0225,0.0165,-0.03,-0.0432
25,-0.0407,0.0125,-0.0828,0.0204,-0.046,-0.0265


,0.03,0.25,0.5,0.75,0.97
1,-0.2354,-0.0664,0.0246,0.1186,0.2808
5,-0.085,-0.0007,0.0467,0.0931,0.1774
15,-0.0214,0.0271,0.054,0.0803,0.1296
25,-0.0012,0.0358,0.0561,0.0767,0.1134


,0.03,0.25,0.5,0.75,0.97
1,-0.0727,0.0163,0.0632,0.112,0.1993
5,-0.0516,-0.0053,0.0205,0.0457,0.0906
15,-0.0279,0.0007,0.0166,0.0323,0.0606
25,-0.0192,0.0039,0.0167,0.0294,0.0526


,0.03,0.25,0.5,0.75,0.97
1,-0.2864,-0.0946,0.0108,0.1209,0.311
5,-0.1354,-0.03,0.0271,0.0856,0.1889
15,-0.0744,-0.008,0.0284,0.0646,0.1305
25,-0.0528,-0.0017,0.0277,0.0568,0.1071


,0.03,0.25,0.5,0.75,0.97
1,-0.4336,-0.1531,0.0022,0.1627,0.4416
5,-0.216,-0.0709,0.01,0.0874,0.2307
15,-0.1132,-0.0281,0.0192,0.0675,0.1493
25,-0.08,-0.0134,0.0233,0.0597,0.1259


,0.03,0.25,0.5,0.75,0.97
1,-0.0251,-0.0034,0.0085,0.0208,0.0421
5,-0.0219,-0.0094,-0.0024,0.0043,0.0163
15,-0.015,-0.0075,-0.0033,0.0012,0.0089
25,-0.0128,-0.0066,-0.0032,0.0003,0.0066


,0.03,0.25,0.5,0.75,0.97
1,-17.422,-17.0958,-16.9131,-16.7288,-16.3912
5,-17.8001,-17.097,-16.7066,-16.3265,-15.6554
15,-17.6009,-16.8929,-16.4744,-16.0676,-15.3429
25,-17.3852,-16.7487,-16.3795,-16.0227,-15.3577


In [12]:
start = Date(1989, 01, 01)
end_d = Date(2024, 01, 31)
assets_names = [:USA_Net_T_Bill, :World_ex_USA_T_Bill, :EM_Net_T_Bill]
data = to(from(merge(excess_returns[assets_names], real_t_bill, real_10Y, dy), start),end_d)
assets_names = colnames(data)
T, n_assets = size(data)
model = VARModel(data)
model_summary(model)

,const,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,Dividend_Yeld,std
USA_Net_T_Bill,0.1683,0.125,0.0203,-0.0207,2.4105,-4.0492,0.035,0.0724
World_ex_USA_T_Bill,-0.0243,0.0265,0.0179,0.0576,0.2398,-1.2235,-0.0083,0.0845
EM_Net_T_Bill,0.2286,-0.2143,0.2693,0.0241,1.3845,-1.6515,0.0528,0.1275
T_Bill_CPI,0.0276,-0.0004,0.0095,-0.0027,0.9043,-0.6602,0.0064,0.0081
Yeld_10Y_CPI,0.0296,-0.0021,0.0084,-0.0014,-0.0064,0.2439,0.0068,0.008
Dividend_Yeld,-0.2241,0.0246,-0.1051,0.0344,-3.7336,2.7902,0.9473,0.0736


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,Dividend_Yeld
USA_Net_T_Bill,1.0,0.8,0.65,-0.14,-0.11,-0.59
World_ex_USA_T_Bill,0.8,1.0,0.74,-0.26,-0.24,-0.48
EM_Net_T_Bill,0.65,0.74,1.0,-0.26,-0.22,-0.32
T_Bill_CPI,-0.14,-0.26,-0.26,1.0,0.99,0.08
Yeld_10Y_CPI,-0.11,-0.24,-0.22,0.99,1.0,0.07
Dividend_Yeld,-0.59,-0.48,-0.32,0.08,0.07,1.0


In [13]:
scenarios = simulate(model, 100, 10_000)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.03, .25, .5, .75, .97], string.(periods), string.(assets_names[a]))
end  

,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,Dividend_Yeld
1,0.073,0.0267,0.0178,0.0221,0.0117,-16.9984
5,0.0805,0.0427,0.0074,0.0072,0.0046,-17.1233
15,0.0708,0.0402,0.0085,0.0013,0.0051,-16.9549
25,0.0683,0.0371,0.0122,0.0001,0.0062,-16.8041


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,Dividend_Yeld
1,0.136,0.1604,0.2324,0.017,0.0162,0.2662
5,0.0693,0.0861,0.1173,0.0115,0.0097,0.5624
15,0.0404,0.0538,0.0709,0.0077,0.0075,0.6962
25,0.0311,0.0423,0.0563,0.006,0.0067,0.668


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,Dividend_Yeld
1,-0.0459,-0.0412,-0.0412,-0.0438,-0.0363,0.0174
5,0.0123,0.005,-0.0042,0.0072,0.0318,-0.0026
15,-0.0184,0.0061,-0.001,0.0009,0.0461,-0.0136
25,-0.0098,-0.0029,-0.0189,0.0296,0.0148,-0.0003


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,Dividend_Yeld
1,0.0105,0.0384,0.0226,-0.0367,-0.0467,-0.0198
5,-0.0409,-0.0085,0.0188,0.0495,-0.0484,0.0374
15,-0.042,-0.0113,-0.0111,0.0212,0.0227,-0.0006
25,0.0005,0.0301,-0.0646,0.0822,0.0185,0.0263


,0.03,0.25,0.5,0.75,0.97
1,-0.184,-0.0175,0.0736,0.1644,0.3289
5,-0.0483,0.0334,0.08,0.1271,0.2122
15,-0.0056,0.0434,0.0711,0.0982,0.1454
25,0.0101,0.0469,0.0682,0.0895,0.1267


,0.03,0.25,0.5,0.75,0.97
1,-0.275,-0.0807,0.0279,0.1349,0.3239
5,-0.1199,-0.0159,0.0426,0.1012,0.2059
15,-0.0601,0.0039,0.0399,0.0762,0.1415
25,-0.0423,0.0092,0.0373,0.0658,0.1162


,0.03,0.25,0.5,0.75,0.97
1,-0.4289,-0.1373,0.0196,0.1746,0.4532
5,-0.2143,-0.0713,0.0077,0.0855,0.2265
15,-0.1252,-0.0393,0.0081,0.0566,0.1407
25,-0.0933,-0.0259,0.012,0.0506,0.1191


,0.03,0.25,0.5,0.75,0.97
1,-0.0103,0.0107,0.0222,0.0337,0.0536
5,-0.014,-0.0006,0.007,0.015,0.0288
15,-0.0131,-0.0039,0.0013,0.0063,0.0158
25,-0.011,-0.004,0.0001,0.0042,0.0115


,0.03,0.25,0.5,0.75,0.97
1,-0.0194,0.0008,0.0117,0.0228,0.0421
5,-0.0134,-0.002,0.0045,0.0112,0.0227
15,-0.0089,-0.0001,0.0051,0.0102,0.0194
25,-0.0063,0.0017,0.0062,0.0108,0.0187


,0.03,0.25,0.5,0.75,0.97
1,-17.4976,-17.1784,-16.9979,-16.8215,-16.498
5,-18.1896,-17.5017,-17.1254,-16.7425,-16.0503
15,-18.2658,-17.4192,-16.953,-16.4862,-15.6561
25,-18.0595,-17.2583,-16.7995,-16.3533,-15.5579


In [14]:
start = Date(1989, 01, 01)
end_d = Date(2025, 01, 31)
assets_names = [:USA_Net_T_Bill, :World_ex_USA_T_Bill, :EM_Net_T_Bill]
data = to(from(merge(excess_returns[assets_names],  real_t_bill, real_10Y, cpi), start),end_d)
assets_names = colnames(data)
T, n_assets = size(data)
model = VARModel(data)
model_summary(model)

,const,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,CPI,std
USA_Net_T_Bill,0.0297,0.114,-0.0109,0.002,1.9133,-3.3474,-0.1946,0.0728
World_ex_USA_T_Bill,0.0304,0.0177,0.0198,0.076,1.0244,-3.4566,-1.9852,0.0836
EM_Net_T_Bill,0.0209,-0.214,0.214,0.0565,0.2529,-0.538,-0.5913,0.1272
T_Bill_CPI,-0.0068,0.0037,0.0063,-0.0093,0.631,0.2691,0.8674,0.0074
Yeld_10Y_CPI,-0.0063,0.0021,0.0048,-0.0076,-0.2822,1.1642,0.8476,0.0073
CPI,0.0069,-0.005,-0.0041,0.0089,0.3303,-0.2685,0.1113,0.0076


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,CPI
USA_Net_T_Bill,1.0,0.79,0.65,-0.13,-0.09,0.15
World_ex_USA_T_Bill,0.79,1.0,0.73,-0.25,-0.22,0.27
EM_Net_T_Bill,0.65,0.73,1.0,-0.25,-0.21,0.27
T_Bill_CPI,-0.13,-0.25,-0.25,1.0,0.99,-0.99
Yeld_10Y_CPI,-0.09,-0.22,-0.21,0.99,1.0,-0.99
CPI,0.15,0.27,0.27,-0.99,-0.99,1.0


In [15]:
scenarios = simulate(model, 100, 10_000)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.03, .25, .5, .75, .97], string.(periods), string.(assets_names[a]))
end  

,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,CPI
1,0.095,0.0471,0.0601,0.023,0.0201,0.0263
5,0.0904,0.0262,0.0486,0.013,0.017,0.0294
15,0.0816,0.0258,0.0514,0.005,0.0151,0.0279
25,0.0799,0.0286,0.0531,0.0015,0.0135,0.0273


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,CPI
1,0.1356,0.1559,0.2323,0.013,0.013,0.0137
5,0.0707,0.0786,0.1192,0.0082,0.0087,0.0074
15,0.043,0.0458,0.0698,0.0079,0.0072,0.0048
25,0.0331,0.0353,0.054,0.0073,0.0062,0.0039


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,CPI
1,-0.0083,0.0389,0.008,-0.0034,-0.0134,-0.0175
5,0.0306,0.0246,0.0266,0.008,0.0148,0.005
15,0.0137,0.0074,0.0035,0.0077,-0.0311,-0.0305
25,0.0104,-0.0021,-0.0059,0.0084,-0.0042,-0.0042


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,CPI
1,-0.0355,0.0518,0.1037,-0.0251,0.0187,-0.0084
5,-0.03,0.0369,0.0754,0.0337,0.1237,0.1099
15,-0.0283,0.0198,0.0091,0.0394,0.0491,0.0732
25,-0.0478,0.0001,-0.0284,0.0085,-0.06,0.0856


,0.03,0.25,0.5,0.75,0.97
1,-0.1607,0.0022,0.0952,0.1876,0.3492
5,-0.0398,0.0423,0.0892,0.1383,0.2243
15,0.0004,0.0526,0.0817,0.1105,0.1633
25,0.0178,0.0574,0.0797,0.1025,0.1419


,0.03,0.25,0.5,0.75,0.97
1,-0.2435,-0.0559,0.0454,0.1495,0.3434
5,-0.1181,-0.0274,0.0253,0.079,0.1755
15,-0.059,-0.0054,0.026,0.0565,0.1119
25,-0.037,0.0047,0.0285,0.0527,0.095


,0.03,0.25,0.5,0.75,0.97
1,-0.3787,-0.0962,0.0627,0.215,0.5013
5,-0.1771,-0.0304,0.0473,0.1284,0.2753
15,-0.0788,0.0041,0.0514,0.0979,0.1828
25,-0.0478,0.0164,0.0533,0.089,0.1551


,0.03,0.25,0.5,0.75,0.97
1,-0.0013,0.0141,0.0229,0.0319,0.0478
5,-0.0024,0.0074,0.0129,0.0185,0.0284
15,-0.0098,-0.0003,0.005,0.0104,0.02
25,-0.0121,-0.0035,0.0015,0.0064,0.0153


,0.03,0.25,0.5,0.75,0.97
1,-0.0043,0.0112,0.02,0.0288,0.0445
5,0.0007,0.0112,0.0171,0.0228,0.0336
15,0.0015,0.0102,0.0151,0.0199,0.0285
25,0.002,0.0093,0.0135,0.0178,0.0252


,0.03,0.25,0.5,0.75,0.97
1,0.0001,0.0171,0.0265,0.0356,0.0517
5,0.0154,0.0245,0.0294,0.0343,0.0432
15,0.0188,0.0247,0.0279,0.0312,0.0369
25,0.0198,0.0246,0.0272,0.0299,0.0345


In [16]:
term = data_source[:Yeld_10Y]   .- data_source[:T_Bill] 
delta_t_bill = diff(data_source[:T_Bill])

start = Date(1989, 01, 01)
end_d = Date(2025, 01, 31)
assets_names = [:USA_Net_T_Bill, :World_ex_USA_T_Bill, :EM_Net_T_Bill]
data = to(from(merge(excess_returns[assets_names],  real_t_bill, delta_t_bill, term, cpi, 1 ./ data_source[:CAPE] ), start),end_d)
assets_names = colnames(data)
T, n_assets = size(data)
model = VARModel(data)
model_summary(model)


,const,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,T_Bill,Yeld_10Y_T_Bill,CPI,CAPE,std
USA_Net_T_Bill,0.0072,0.1263,0.005,-0.008,-1.8302,0.5984,-1.1308,-0.6101,0.7496,0.073
World_ex_USA_T_Bill,0.0296,0.0195,-0.0054,0.0788,-2.2932,2.2974,-0.7303,-2.0977,0.0107,0.0836
EM_Net_T_Bill,-0.0371,-0.1826,0.2767,0.0279,-1.5388,-0.3012,-1.0947,-1.6889,1.9833,0.1271
T_Bill_CPI,-0.0086,0.0048,0.0058,-0.01,0.8656,0.2239,0.0435,0.8123,0.0632,0.0073
T_Bill,0.0028,-0.0065,0.0017,0.0001,-0.0883,0.5068,0.0603,-0.0831,-0.08,0.0038
Yeld_10Y_T_Bill,0.0,-0.0058,0.002,0.0055,-0.1397,-0.579,0.836,-0.0776,0.0718,0.0042
CPI,0.0093,-0.0064,-0.0054,0.01,0.1134,-0.1013,-0.0289,0.1688,-0.0823,0.0076
CAPE,0.0023,-0.0006,-0.0033,0.0001,0.0347,-0.0672,0.0539,0.09,0.9001,0.003


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,T_Bill,Yeld_10Y_T_Bill,CPI,CAPE
USA_Net_T_Bill,1.0,0.8,0.65,-0.14,0.16,0.27,0.16,-0.61
World_ex_USA_T_Bill,0.8,1.0,0.75,-0.27,0.16,0.28,0.28,-0.48
EM_Net_T_Bill,0.65,0.75,1.0,-0.27,0.19,0.35,0.28,-0.33
T_Bill_CPI,-0.14,-0.27,-0.27,1.0,-0.19,-0.12,-0.99,-0.01
T_Bill,0.16,0.16,0.19,-0.19,1.0,-0.21,0.3,-0.06
Yeld_10Y_T_Bill,0.27,0.28,0.35,-0.12,-0.21,1.0,0.09,-0.08
CPI,0.16,0.28,0.28,-0.99,0.3,0.09,1.0,0.0
CAPE,-0.61,-0.48,-0.33,-0.01,-0.06,-0.08,0.0,1.0


In [17]:
scenarios = simulate(model, 100, 10_000)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.03, .25, .5, .75, .97], string.(periods), string.(assets_names[a]))
end  

,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,T_Bill,Yeld_10Y_T_Bill,CPI,CAPE
1,0.0767,0.0311,0.0233,0.0215,-0.0043,-0.0117,0.0277,0.1177
5,0.0693,0.0124,0.0073,0.0105,-0.0032,0.0228,0.031,0.1284
15,0.0691,0.0179,0.0211,0.0047,-0.0017,0.0409,0.0295,0.1396
25,0.0711,0.0229,0.0264,0.0026,-0.001,0.0449,0.0288,0.1417


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,T_Bill,Yeld_10Y_T_Bill,CPI,CAPE
1,0.1374,0.157,0.2314,0.0133,0.0091,0.0165,0.0138,0.0107
5,0.0673,0.0777,0.1106,0.0098,0.004,0.0266,0.0077,0.0199
15,0.0381,0.0436,0.065,0.0095,0.0014,0.0221,0.0052,0.0187
25,0.0292,0.0336,0.0506,0.0084,0.0009,0.0183,0.0042,0.0157


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,T_Bill,Yeld_10Y_T_Bill,CPI,CAPE
1,-0.0269,0.0115,0.0126,0.0195,-0.0014,-0.0067,-0.0328,0.0244
5,0.0333,0.0125,-0.0219,-0.023,-0.0232,-0.002,-0.0222,-0.0093
15,-0.0289,-0.0118,-0.0172,-0.0216,0.0393,-0.0144,-0.0106,-0.0108
25,-0.0184,0.0138,0.0352,-0.0089,0.0298,0.0154,-0.0191,0.0288


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,T_Bill,Yeld_10Y_T_Bill,CPI,CAPE
1,0.0302,0.0305,-0.0009,0.0186,0.0305,-0.0407,-0.0263,-0.0131
5,-0.1115,0.0045,-0.1,-0.0351,-0.0239,0.0087,-0.048,-0.0072
15,0.038,0.0262,0.0116,0.0559,0.0025,-0.0091,0.0079,0.0181
25,0.1564,0.0109,0.0206,0.0831,0.0251,0.0451,0.0595,0.006


,0.03,0.25,0.5,0.75,0.97
1,-0.1883,-0.0156,0.0768,0.1693,0.3314
5,-0.0559,0.0239,0.0684,0.1157,0.1975
15,-0.0039,0.0437,0.0691,0.0947,0.1402
25,0.0158,0.0518,0.0714,0.0907,0.126


...

,0.03,0.25,0.5,0.75,0.97
1,-0.003,0.0124,0.0216,0.0303,0.0468
5,-0.0081,0.004,0.0105,0.017,0.0288
15,-0.0136,-0.0016,0.0048,0.011,0.0226
25,-0.0133,-0.003,0.0025,0.0083,0.0184


,0.03,0.25,0.5,0.75,0.97
1,-0.0214,-0.0103,-0.0043,0.0018,0.0129
5,-0.0108,-0.0059,-0.0032,-0.0005,0.0041
15,-0.0043,-0.0026,-0.0017,-0.0007,0.0011
25,-0.0026,-0.0016,-0.001,-0.0004,0.0006


,0.03,0.25,0.5,0.75,0.97
1,-0.0427,-0.0229,-0.0116,-0.0005,0.0194
5,-0.0273,0.0046,0.0229,0.0411,0.0723
15,-0.001,0.0258,0.041,0.056,0.0823
25,0.0107,0.0327,0.0449,0.057,0.0795


,0.03,0.25,0.5,0.75,0.97
1,0.0014,0.0185,0.0277,0.0371,0.0533
5,0.0166,0.0258,0.0311,0.0362,0.0454
15,0.0197,0.0259,0.0294,0.033,0.0392
25,0.0208,0.0261,0.0288,0.0316,0.0367


,0.03,0.25,0.5,0.75,0.97
1,0.0976,0.1106,0.1177,0.1248,0.1381
5,0.0907,0.115,0.1285,0.1421,0.1662
15,0.1041,0.127,0.1395,0.1519,0.1748
25,0.1123,0.1313,0.1415,0.1523,0.1716


In [18]:
real_USA = sum(ret_in_years[[1,4],:,:], dims=1)[1, :,:]
print_scenarios_percentiles(real_USA, [.03, .25, .5, .75, .97], string.(periods), "Real USA")

,0.03,0.25,0.5,0.75,0.97
1,-0.1616,0.0073,0.0986,0.1895,0.3496
5,-0.0474,0.0336,0.0794,0.1263,0.2087
15,-0.002,0.0473,0.0742,0.1007,0.1474
25,0.0148,0.0535,0.0739,0.0945,0.1306
